### Data Preprossing------------------------------------

In [59]:
import os   # for interacting with the operating system,  include many functions to interact with the file system
import numpy as np  # Python library used for working with arrays
import pandas as pd  #data manipulation and analysis
import re

In [60]:
# list all files in directory - Raw Data
os.getcwd()

'c:\\Users\\shyam.kalliadan\\Desktop\\ESR10\\programming\\ESR_10'

### 1. Data Column Rename -------------------

In [61]:
raw_data_directory = os.listdir('Raw_data')
processed_data_directory = os.listdir('processed_data')

In [62]:
def mapper(old_name):
    x = re.sub("^(sub_DAQ.mod_LogicLev.cntr)\\_([a-zA-Z0-9]+)\\.([a-zA-Z]+)\\_", "", old_name)
    x = re.sub("\\.(a_var)$", "", x)
    x = re.sub(" and ", "_", x)
    return x

In [63]:
for raw_file in raw_data_directory:
    pbr_dir = re.split("\\_", raw_file)
    pbr_dir = pbr_dir[0]
    new_path = os.path.join('processed_data', pbr_dir)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    fpc_list = os.listdir(new_path)
    if raw_file not in fpc_list:
        raw_file_path = os.path.join('Raw_data', raw_file)
        raw_data_frame = pd.read_csv(raw_file_path, sep=';')
        raw_data = raw_data_frame.rename(columns=mapper)

        save_path = os.path.join(new_path, raw_file)
        raw_data.to_csv(save_path, index = False)
        os.remove(raw_file_path)
        

#### 2. Data Merging and column drop

In [71]:
processed_dir = './processed_data'

species_dir_list = [dir for dir in os.listdir(processed_dir)]
print(species_dir_list)

['FPC11-Rhodomonas-salina', 'FPC12-Diacronema-lutheri']


In [90]:
def merger_function(dir,species_dir):
    print(dir)
    print(species_dir)
    #pd.merge(FPC11_merge1, FPC11_13_12_21_to_31_01_22_flow_and_pressure, on='Date_Time', how='inner')


In [91]:
for dir in species_dir_list:
    species_dir = os.path.join(processed_dir,dir)
    #print(species_dir)
    #print(dir)
    dir = list()
    for species_file in os.listdir(species_dir):
        #print(species_file)
        dir.append(species_file)
    
    #print(dir)
    merger_function(dir,species_dir)


['FPC11-Rhodomonas-salina_12_12_21_to_31_01_22_light_temp_ph.csv', 'FPC11-Rhodomonas-salina_12_12_21_to_31_01_22_productivity_efficiencycsv.csv', 'FPC11-Rhodomonas-salina_13_12_21_to_31_01_22_flow_and_pressure.csv']
./processed_data\FPC11-Rhodomonas-salina
['FPC12-Diacronema-lutheri_24_10_21_to_01_02_22light_temp_ph.csv', 'FPC12-Diacronema-lutheri_24_10_21_to_01_02_flow_and_pressure.csv', 'FPC12-Diacronema-lutheri_24_10_21_to_01_02_productivity_efficency.csv']
./processed_data\FPC12-Diacronema-lutheri
